# Get book characters 

In [ ]:
import pandas as pd
import string
from bs4 import BeautifulSoup
from urllib.request import urlopen

#web scraping list of characters from witcher wiki
#https://witcher.fandom.com/wiki/Category:Characters_in_the_stories
from bs4 import BeautifulSoup
from urllib.request import urlopen

book_characters = {}

def get_characters(book, url):
    book_characters[book] = set()
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")

    images = soup.find_all("img")

    for image in images:
        name = image["alt"]
        name = name.split(" ")[0]
        if name not in book_characters[book]:
            if not name == "Witcher" and not name == "Cirilla":
                book_characters[book].add(name)

# I
get_characters('I' ,"https://witcher.fandom.com/wiki/Category:Blood_of_Elves_characters")
# II
get_characters('II', "https://witcher.fandom.com/wiki/Category:Time_of_Contempt_characters")
# III
get_characters('III', "https://witcher.fandom.com/wiki/Category:Baptism_of_Fire_characters")
# IV
get_characters('IV', "https://witcher.fandom.com/wiki/Category:The_Tower_of_the_Swallow_characters")
# V
get_characters('V', "https://witcher.fandom.com/wiki/Category:The_Lady_of_the_Lake_characters")

In [ ]:
for book, list_characters in book_characters.items():
    print(book)
    print(sorted(list_characters))
    print(len(list_characters))
    print('\n')

# Get chapters from file txt

In [ ]:
import os
books = ['I - Blood of Elves', 'II - Times of Contempt', 'III - Baptism of Fire', 'IV - The Tower of the Swallow', 'V - The Lady of the Lake']
book_chapters = {}

for book in books:
    book_chapters[book] = set()
    chapters = os.listdir('./books/book_chapters/' + book + '/txt/')
    for chapter in chapters:
        book_chapters[book].add(int(chapter.replace("chapter-", "").replace(".txt", "")))

In [ ]:
for book, chapters in book_chapters.items():
    print(book)
    print(chapters)

# Get characters for chapters

In [ ]:
def get_characters_for_chapters(chapter, name_book):
    chapter_characters = set()
    lines = []
    for line in chapter:
        lines.append(line)

    table = str.maketrans('', '', string.punctuation)
    stripped = [l.translate(table) for l in lines]
    
    for line in stripped:
        for charater in book_characters[name_book]:
            if charater in line:
                chapter_characters.add(charater)
                #print(chapter_characters)
        
    return chapter_characters

In [ ]:
def get_column(chapter_characters, name_book):
    l = []
    for character in book_characters[name_book]:
        if character in chapter_characters:
            l.append(1)
        else:
            l.append(0)
    return l

# Create final csv 

In [ ]:
import pandas as pd

for book, list_chapters in book_chapters.items():
    data = {}
    name_book = book.split(" ", 1)[0]
    for chapter in list_chapters:
        path_chapter = "./books/book_chapters/" + book + "/txt/chapter-" + str(chapter) + ".txt"
        data[chapter] = get_column(get_characters_for_chapters(open(path_chapter, "r"), name_book), name_book)
    
    df = pd.DataFrame(data, index=book_characters[name_book])
    
    # remove characters that not appear in 50% of chapters
    for row in df.iterrows():
        if row[1].sum() < len(list_chapters)/2:
            df = df.drop(row[0])

    # change name of columns
    columns = []
    for column in df.columns:
        columns.append('c' + str(column))
    df.columns = columns
    
    # sort df
    df = df.sort_values(by = columns, ascending = False)
    i = 0
    for row in df.iterrows():
        if row[1].sum() == len(list_chapters):
            i += 1
    
    df_c = df.iloc[:i,:]
    df_u = df.iloc[i:int((len(df)+i)/2),:]
    df_u = df_u.sort_values(by = columns, ascending = True)
    df_d = df.iloc[int((len(df)+i)/2):,:]

    df = df_u.append(df_c)
    df = df.append(df_d)

    df.to_csv("./books/book_chapters/" + book + "/final-book-" + name_book + "-sort.csv")    
    

# Create json

In [ ]:
import pandas as pd
import json

for book in books:
    current_book = pd.read_csv("./books/book_chapters/" + book + "/final-book-" + book.split(" ", 1)[0] + "-all.csv")
    dict = {}
    i = 1
    l = []
    for column in current_book.columns[1:]:
        l1 = {}
        l1['chapter'] = int(column.replace('c', ''))
        dict[column] = []
        for row in current_book.iterrows():
            # name = row[1][0]
            l1[row[1][0]] = row[1][i]
            if row[1][i] == 1:
                dict[column].append(row[1][0])
                # l1.append(row[1][0])
        i += 1
        # print(sorted(l1.items(), key=lambda x: x[1], reverse=True))
        
        l.append(l1)
        # print(l)
    with open('./app/assets/book-' + book.split(" ", 1)[0] + '-all.json', 'w') as f:
        json.dump(l, f)
        break


In [ ]:
import json
data_json = json.dumps(l)
print(data)

## Overview

In [ ]:
dict_characters_books = {}
for book in books:
    current_book = pd.read_csv("./books/book_chapters/" + book + "/final-book-" + book.split(" ", 1)[0] + ".csv")
    dict_characters_books[book.split(" ", 1)[0]] = []
    for row in current_book.iterrows():
        dict_characters_books[book.split(" ", 1)[0]].append(row[1][0])
    

In [ ]:
total_characters = set()
for book, l_characters in dict_characters_books.items():
    for c in l_characters:
        total_characters.add(c)

In [ ]:
def get_column_book(book):
    l = []
    for character in total_characters:
        if character in dict_characters_books[book]:
            l.append(1)
        else:
            l.append(0)
    return l
        

In [ ]:
data = {}
for book in books:
    data['b-' + book.split(" ", 1)[0]] = get_column_book(book.split(" ", 1)[0])

df = pd.DataFrame(data, index=list(total_characters))
cols = []
for col in df.columns:
    cols.append(col)

df = df.sort_values(by = cols, ascending = False)

i = 0
for row in df.iterrows():
    if row[1].sum() == len(list_chapters):
        i += 1

df_c = df.iloc[:i, :]
df_u = df.iloc[i:int((len(df)+i)/2), :]
df_u = df_u.sort_values(by=cols, ascending=True)
df_d = df.iloc[int((len(df)+i)/2):, :]

df = df_u.append(df_c)
df = df.append(df_d)


In [ ]:
dict = {}
l = []
i = 0
roman = {'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5}
# print(df.columns[0:])
for column in df.columns[0:]:
    l1 = {}
    l1['book'] = roman[column.replace('b-', '')]
    dict[column] = []
    for row in df.iterrows():
        # name = row[1][0]
        #print(row[1][i])
        l1[row[0]] = int(row[1][i])

    i += 1
    # print(l1)
    l.append(l1)


In [ ]:
with open('./app/assets/books-overview.json', 'w') as f:
    json.dump(l, f)